In [1]:
import json
import sys
import requests
import pandas as pd
from bs4 import BeautifulSoup


def pyynto(url):
    try:
        vastaus = requests.get(url, timeout=1000)
        vastaus.raise_for_status()
        return vastaus.content
    except requests.RequestException as e:
        print(f"Virhe datan haussa: {e}")
        sys.exit(1)

def keitto(html):
    try:
        html_str = html.decode('utf-8')
        soup = BeautifulSoup(html_str,'html.parser')
        data = json.loads(soup.find('script', id='online-match-data').text)
        return(data)
    except AttributeError as e:
        print(f'Virhe: {e}')
        sys.exit(1)

def kokoonpano(data):
    
    kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])
    
    vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])
    
    kotipj = pd.json_normalize(
        data['teams']['home']["managersLineUp"])
    
    vieraspj = pd.json_normalize(
        data['teams']['away']["managersLineUp"])
    
    kokoonpano = pd.concat([kotikokoopano, vieraskokoopano],  ignore_index=False)

def data_muokkaus(data):
    try:

        koti = pd.json_normalize(data['teams']['home'], record_path=['lineUp'],
                                 meta=['id', 'name', 'shorthand', 'icon', 'logo'])

        koti_pelinjohtajat = pd.json_normalize(
            data['teams']['home'], record_path=['managersLineUp'])

        koti_kaarten_väli = pd.json_normalize(data['details'], record_path=[
                                              "innerCircle_home"], meta=['id'], record_prefix='sisäkaari_', errors='ignore')

        kotivalmennus = pd.json_normalize(data['details'], record_path=[
                                          "OtherRoles:home"], record_prefix='valmennus', errors='ignore')

        kotivalmennus.columns = ['valmennuskoti']

        vieras = pd.json_normalize(data['teams']['away'], record_path=['lineUp'],
                                   meta=['id', 'name', 'shorthand', 'icon', 'logo'])

        vieras_pelinjohtajat = pd.json_normalize(
            data['teams']['away'], record_path=['managersLineUp'])

        vieras_kaarten_väli = pd.json_normalize(data['details'], record_path=[
                                                "innerCircle_away"], meta=['id'], record_prefix='sisäkaari_', errors='ignore')

        vierasvalmennus = pd.json_normalize(data['details'], record_path=[
                                            "OtherRoles:away"], record_prefix='valmennus', errors='ignore')

        vierasvalmennus.columns = ['valmennusvieras']

        kotijoukkue_pelinjohto = pd.merge(
            koti_pelinjohtajat, koti_kaarten_väli, left_on='manager.id', right_on='sisäkaari_id', how='outer')

        vierasjoukkue_pelinjohto = pd.merge(
            vieras_pelinjohtajat, vieras_kaarten_väli, left_on='manager.id', right_on='sisäkaari_id', how='outer')

        koti_kotiutuslyönti = pd.json_normalize(
            data['details']["scoringContestPlayers:0"], record_path=['home'], record_prefix='koti_')

        vieras_kotiutuslyönti = pd.json_normalize(
            data['details']["scoringContestPlayers:0"], record_path=['away'], record_prefix='vieras_')

        koti_kotiutuslyöntijatkot = pd.json_normalize(
            data['details']["scoringContestPlayers:1"], record_path=['home'], record_prefix='kotijatko_')

        vieras_kotiutuslyöntijatkot = pd.json_normalize(
            data['details']["scoringContestPlayers:1"], record_path=['away'], record_prefix='vierasjatko_')

        muut_tiedot = pd.json_normalize(data['details'], errors='ignore')

        kokoonpano = pd.concat([koti, vieras],  ignore_index=False)

        kotarit = pd.concat([koti_kotiutuslyönti, vieras_kotiutuslyönti, koti_kotiutuslyöntijatkot,
                            vieras_kotiutuslyöntijatkot],  axis=1, ignore_index=False)

        pelinjohto = pd.concat(
            [kotijoukkue_pelinjohto, vierasjoukkue_pelinjohto], ignore_index=False)

        valmennus = pd.concat([kotivalmennus, vierasvalmennus],
                              axis=1, join='outer', ignore_index=False)

        return (kokoonpano, kotarit, pelinjohto, valmennus, muut_tiedot)
    except (TypeError, KeyError) as e:
        print(f'Virhe {e}')


def main():

    ottelunUrl = ('https://v1.pesistulokset.fi/media/ottelu/107013')

    hae_data = pyynto(ottelunUrl)

    tiedot = keitto(hae_data)
    
    global data
    
    data = tiedot
    
    

main()

In [ ]:
data

In [9]:
kotikokoopano = pd.json_normalize(data['teams']['home']["managersLineUp"])

In [107]:
kotikokoopano["nimi"] = data['teams']['home']["shorthand"]

In [2]:
koti = data["details"]["scoringContestPlayers:0"]["home"]

In [7]:
koti

[{'batter': 3, 'runner': 11},
 {'batter': 10, 'runner': 1},
 {'batter': 4, 'runner': 7},
 {'batter': 12, 'runner': 6},
 {'batter': 2, 'runner': 5}]

In [3]:
vieras = data["details"]["scoringContestPlayers:0"]["away"]

In [4]:
vierasjatko = data["details"]["scoringContestPlayers:1"]["away"]

In [6]:
kotijatko = data["details"]["scoringContestPlayers:1"]["home"]

In [18]:
vieras = data["details"]["innerCircle_away"]

In [22]:
vaihtopenkki = {"koti": data["details"]["OtherRoles:home"],
                "vieras": data["details"]["OtherRoles:away"]}

In [20]:
rooli = pd.concat([pd.json_normalize(koti), pd.json_normalize(vieras)])

In [40]:
kotari = pd.concat([pd.json_normalize(koti), pd.json_normalize(kotijatko), pd.json_normalize(vieras), pd.json_normalize(vierasjatko)])

In [67]:
kotikotari = pd.concat([pd.json_normalize(koti), pd.json_normalize(
    kotijatko)])

In [97]:
vieraskotari = pd.concat([pd.json_normalize(vieras), pd.json_normalize(
    vierasjatko)])

In [2]:
kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])
    
vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])
    
kotipj = pd.json_normalize(
        data['teams']['home']["managersLineUp"])
    
vieraspj = pd.json_normalize(
        data['teams']['away']["managersLineUp"])
    
kokoonpano = pd.concat([kotikokoopano, vieraskokoopano],  ignore_index=False)

In [3]:
kotikokoopano

,originalNumber,defensivePosition,prizes,trophies,player.id,player.firstName,player.lastName,player.url,player.mainId,player.image,player.age,player.birthday,player.birthday_today
0,1,6.0,[],"[{'id': 1, 'name': 'Kuukauden pelaaja', 'detai...",8878,Severi,Tikkakoski,https://v1.pesistulokset.fi/pelaaja/8878,8878,https://www.pesistulokset.fi/storage/pelaajaku...,25,11.3.2000,False
1,2,4.0,[],"[{'id': 40, 'name': 'SM-hopeamitali', 'detail'...",6090,Juuso,Myllyniemi,https://v1.pesistulokset.fi/pelaaja/6090,6090,https://www.pesistulokset.fi/storage/pelaajaku...,34,11.6.1991,False
2,3,5.0,[],"[{'id': 41, 'name': 'SM-pronssimitali', 'detai...",8440,Elmeri,Lieto,https://v1.pesistulokset.fi/pelaaja/8440,8440,https://www.pesistulokset.fi/storage/pelaajaku...,25,14.1.2000,False
3,4,9.0,[],"[{'id': 2, 'name': 'Vuoden pesäpalloilija', 'd...",1315,Henri,Puputti,https://v1.pesistulokset.fi/pelaaja/1315,1315,https://www.pesistulokset.fi/storage/pelaajaku...,41,26.7.1983,False
4,5,2.0,[],"[{'id': 40, 'name': 'SM-hopeamitali', 'detail'...",9542,Tomi,Lehtonen,https://v1.pesistulokset.fi/pelaaja/9542,9542,https://www.pesistulokset.fi/storage/pelaajaku...,23,3.8.2001,False
5,6,3.0,[],"[{'id': 8, 'name': 'Vuoden tulokas, miesten Su...",6640,Simo,Vainikainen,https://v1.pesistulokset.fi/pelaaja/6640,6640,https://www.pesistulokset.fi/storage/pelaajaku...,32,20.12.1992,False
6,7,7.0,[],"[{'id': 31, 'name': 'Vuoden poikalukkari', 'de...",8270,Kalle-Tapio,Huusko,https://v1.pesistulokset.fi/pelaaja/8270,8270,https://www.pesistulokset.fi/storage/pelaajaku...,26,26.2.1999,False
7,8,1.0,"[{'id': 5, 'name': 'I', 'status': 2, 'details'...","[{'id': 2, 'name': 'Vuoden pesäpalloilija', 'd...",2708,Juha,Puhtimäki,https://v1.pesistulokset.fi/pelaaja/2708,2708,https://www.pesistulokset.fi/storage/pelaajaku...,39,20.6.1985,False
8,9,8.0,[],"[{'id': 40, 'name': 'SM-hopeamitali', 'detail'...",7084,Topias,Lilja,https://v1.pesistulokset.fi/pelaaja/7084,7084,https://www.pesistulokset.fi/storage/pelaajaku...,28,31.10.1996,False
9,10,NaN,[],"[{'id': 12, 'name': 'Kultainen maila, miehet',...",8574,Perttu,Ruuska,https://v1.pesistulokset.fi/pelaaja/8574,8574,https://www.pesistulokset.fi/storage/pelaajaku...,24,15.2.2001,False


In [74]:
kotikotaril = vieraskotari[["runner", "batter"]]

In [98]:
vieraskotari = vieraskotari[["runner", "batter"]]

In [56]:
kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])

vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])

kotikokoopano["joukkue"] = data['teams']['home']["shorthand"]

vieraskokoopano["joukkue"] = data['teams']['away']["shorthand"]
    
kotikokoopano["etenijänimi"] = kotikokoopano["player.firstName"] + \
        " " + kotikokoopano["player.lastName"]
        
kotikokoopano["lyöjänimi"] = kotikokoopano["player.firstName"] + \
        " " + kotikokoopano["player.lastName"]
        
vieraskokoopano["etenijänimi"] = vieraskokoopano["player.firstName"] + \
        " " + vieraskokoopano["player.lastName"]

vieraskokoopano["lyöjänimi"] = vieraskokoopano["player.firstName"] + \
        " " + vieraskokoopano["player.lastName"]
    
kotieka = data["details"]["scoringContestPlayers:0"]["home"]
    
vieraseka = data["details"]["scoringContestPlayers:0"]["away"]
    
kotijatko = data["details"]["scoringContestPlayers:1"]["home"]
    
vierasjatko = data["details"]["scoringContestPlayers:1"]["away"]
    
kotikotari = pd.concat([pd.json_normalize(kotieka), pd.json_normalize(
        kotijatko)])
    
vieraskotari = pd.concat([pd.json_normalize(vieraseka), pd.json_normalize(
        vierasjatko)])
    
kotikotarieka = kotikotari.merge(kotikokoopano[[
        "etenijänimi", "joukkue", "originalNumber"]], left_on="runner", right_on="originalNumber")
    
kotikotarilopullinen = kotikotarieka.merge(kotikokoopano[[
    "lyöjänimi", "joukkue", "originalNumber"]], left_on="batter", right_on="originalNumber", suffixes=("_etenijä", "_lyöja"))
    
vieraskotarieka = vieraskotari.merge(vieraskokoopano[[
    "etenijänimi", "joukkue", "originalNumber"]], left_on="runner", right_on="originalNumber")

vieraskotarilopullinen = vieraskotarieka.merge(vieraskokoopano[[
    "lyöjänimi", "joukkue", "originalNumber"]], left_on="batter", right_on="originalNumber", suffixes=("_etenijä", "_lyöja"))
    
pd.concat([kotikotarilopullinen, vieraskotarilopullinen])

,batter,runner,etenijänimi,joukkue_etenijä,originalNumber_etenijä,lyöjänimi,joukkue_lyöja,originalNumber_lyöja
0,10,1,Severi Tikkakoski,Manse,1,Perttu Ruuska,Manse,10
1,4,2,Juuso Myllyniemi,Manse,2,Henri Puputti,Manse,4
2,3,6,Simo Vainikainen,Manse,6,Elmeri Lieto,Manse,3
3,8,12,Timo Torppa,Manse,12,Juha Puhtimäki,Manse,8
4,7,5,Tomi Lehtonen,Manse,5,Kalle-Tapio Huusko,Manse,7
5,10,1,Severi Tikkakoski,Manse,1,Perttu Ruuska,Manse,10
6,4,2,Juuso Myllyniemi,Manse,2,Henri Puputti,Manse,4
7,3,6,Simo Vainikainen,Manse,6,Elmeri Lieto,Manse,3
0,10,2,Jere Vikström,SoJy,2,Roope Korhonen,SoJy,10
1,3,12,Elmeri Purmonen,SoJy,12,Hannes Pekkinen,SoJy,3


In [12]:
eka = pd.json_normalize(kotieka)

In [14]:
eka["kierros"] = 1

In [15]:
eka

,batter,runner,kierros
0,3,11,1
1,10,1,1
2,4,7,1
3,12,6,1
4,2,5,1


In [57]:
kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])
    
kotikokoopano["joukkue"] = data['teams']['home']["shorthand"]
    
vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])
    
vieraskokoopano["joukkue"] = data['teams']['away']["shorthand"]

In [58]:

kotikokoopano["nimi"] = kotikokoopano["player.firstName"] + \
    " " + kotikokoopano["player.lastName"]

kotikokoopano["lyöjänimi"] = kotikokoopano["player.firstName"] + \
    " " + kotikokoopano["player.lastName"]

vieraskokoopano["nimi"] = vieraskokoopano["player.firstName"] + \
    " " + vieraskokoopano["player.lastName"]

vieraskokoopano["lyöjänimi"] = vieraskokoopano["player.firstName"] + \
    " " + vieraskokoopano["player.lastName"]

kotieka = pd.json_normalize(data["details"]["scoringContestPlayers:0"]["home"])

kotieka["kierros"] = 1

vieraseka = pd.json_normalize(
    data["details"]["scoringContestPlayers:0"]["away"])

vieraseka["kierros"] = 1

kotijatko = pd.json_normalize(
    data["details"]["scoringContestPlayers:1"]["home"])

kotijatko["kierros"] = 2

vierasjatko = pd.json_normalize(
    data["details"]["scoringContestPlayers:1"]["away"])

vierasjatko["kierros"] = 2

kotikotari = pd.concat([kotieka, kotijatko])

vieraskotari = pd.concat([vieraseka, vierasjatko])

In [59]:
kotikotarieka = kotikotari.merge(kotikokoopano[[
    "nimi", "originalNumber"]], left_on="batter", right_on="originalNumber", how="inner", validate="many_to_many")

kotikotarilopullinen = kotikotarieka.merge(kotikokoopano[[
    "nimi", "originalNumber", "joukkue"]], left_on="runner", right_on="originalNumber", suffixes=("_lyöjä", "_etenijä"), how="inner", validate="many_to_many")

vieraskotarieka = vieraskotari.merge(vieraskokoopano[[
    "nimi", "originalNumber"]], left_on="batter", right_on="originalNumber", how="inner", validate="many_to_many")

vieraskotarilopullinen = vieraskotarieka.merge(vieraskokoopano[[
    "nimi", "originalNumber", "joukkue"]], left_on="runner", right_on="originalNumber", suffixes=("_lyöjä", "_etenijä"), how="inner", validate="many_to_many")

pd.concat([kotikotarilopullinen, vieraskotarilopullinen])

,batter,runner,kierros,nimi_lyöjä,originalNumber_lyöjä,nimi_etenijä,originalNumber_etenijä,joukkue
0,10,1,1,Perttu Ruuska,10,Severi Tikkakoski,1,Manse
1,4,2,1,Henri Puputti,4,Juuso Myllyniemi,2,Manse
2,3,6,1,Elmeri Lieto,3,Simo Vainikainen,6,Manse
3,8,12,1,Juha Puhtimäki,8,Timo Torppa,12,Manse
4,7,5,1,Kalle-Tapio Huusko,7,Tomi Lehtonen,5,Manse
5,10,1,2,Perttu Ruuska,10,Severi Tikkakoski,1,Manse
6,4,2,2,Henri Puputti,4,Juuso Myllyniemi,2,Manse
7,3,6,2,Elmeri Lieto,3,Simo Vainikainen,6,Manse
0,10,2,1,Roope Korhonen,10,Jere Vikström,2,SoJy
1,3,12,1,Hannes Pekkinen,3,Elmeri Purmonen,12,SoJy


In [ ]:
"""query TulosvetoDrawDetailsQuery($drawId: ID!, $lang: String) {  scoreDraws(ids: [$drawId], lang: $lang) {    id    betSlipId    closeTime    row {      name      id      eventId      sportId      outcome {        home        away      }      score {        min        max      }      event {        tournament {          name          id        }        externalId      }    }    popularity {      exchange      home {        score        percentage      }      away {        score        percentage      }    }    gameRuleSet {      oddsType      minStake      maxStake      maxSelections      maxPrice    }    resultsAvailableTime    jackpots {      type    }  }  tulosvetoOdds(drawId: $drawId) {    timestamp    exchange    odds {      value      baseValue      selections {        homeScores        awayScores      }    }  }}
"""